In [2]:
import findspark
findspark.init('/home/aditya/spark-3.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Lreg").getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [44]:
mydf = df.select('Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
 'Fare', 'Embarked')
myfinaldf = mydf.na.drop()

In [62]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorIndexer, VectorAssembler
# dfnew = dfnew.withColumn('cabinfloor',substring('Cabin',1,1))
# dfnew = dfnew.na.fill('Z', subset='cabinfloor')
# dfnew.groupby('cabinfloor').count().show()
# indexer = StringIndexer(inputCol="cabinfloor", outputCol="cabinfloor_lineIndex")
# indexed = indexer.fit(dfnew).transform(dfnew)
gender_indexer = StringIndexer(inputCol="Sex",outputCol="IndexedSex")
gender_encoder = OneHotEncoder(inputCol="IndexedSex", outputCol="SexVec")

Embarked_indexer = StringIndexer(inputCol="Embarked",outputCol="IndexedEmbarked")
Embarked_encoder = OneHotEncoder(inputCol="IndexedEmbarked", outputCol="EmbarkedVec")

assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 
 'Fare', 'EmbarkedVec'], outputCol='features')

In [63]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [64]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Survived')

In [65]:
mlpipeline = Pipeline(stages=[gender_indexer,gender_encoder,Embarked_indexer,
                             Embarked_encoder,assembler, log_reg])

In [66]:
##split data into train and test
train_data, test_data = myfinaldf.randomSplit([0.7,0.3])

In [67]:
mymodel = mlpipeline.fit(train_data)

In [72]:
results = mymodel.transform(test_data)

In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [77]:
myeval = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",
                                      labelCol='Survived')

In [78]:
myeval.evaluate(results)

0.8505549682875256

In [28]:
## evaluationg the model

In [38]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [42]:
my_evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')

In [44]:
my_final_roc = my_evaluator.evaluate(test_results.predictions)

In [45]:
my_final_roc

1.0